In [3]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from TimeSeriesLearningUtils import TimeSeriesDataset, get_data

In [5]:
CURRENCY_LST = ['BTC', 'ETH', 'LTC']
N_CLASSES = 3
FREQUENCY = "D"
WINDOW_SIZE = 50
NEUTRAL_QUANTILE = 0.33

In [6]:
X, y, features, dfs = get_data(currency_lst= CURRENCY_LST,
                               data_frequency='1d',
                               pred_frequency=FREQUENCY,
                               n_classes= N_CLASSES, 
                               window_size=  WINDOW_SIZE,
                               neutral_quantile = NEUTRAL_QUANTILE,
                               log_price=True,
                               remove_trend=True,
                               ma_period=7,
                               include_indicators = False,
                               include_imfs = False,
                               open_high_low_volume = False
                               )

In [7]:
(X.shape, y.shape)

((3, 1235, 1), (3, 1235))

In [8]:
dfs['BTC']

,change_dir,close,pct_diff
Timestamp,,,
2017-08-24,1,0.047386,0.005760
2017-08-25,2,0.033297,-0.000982
2017-08-26,1,0.039814,0.001575
2017-08-27,2,0.025855,-0.000757
2017-08-28,1,0.030877,0.002107
...,...,...,...
2021-05-03,2,0.019363,0.000950
2021-05-04,0,-0.047812,-0.006569
2021-05-05,1,0.022211,0.007041


In [9]:
#the first WINDOW_SIZE values of change direction will be seen as NaN 

dfs['BTC']['change_dir'].value_counts() / len(dfs['BTC'])

1    0.367332
2    0.330377
0    0.302291
Name: change_dir, dtype: float64

In [8]:
N_CURRENCIES = 3
INPUT_FEATURE_SIZE = 1
WINDOW_SIZE = 50
TRAIN_PERCENTAGE, VAL_PERCENTAGE, TEST_PERCENTAGE = 0.70, 0.15, 0.15

In [10]:
#datasets for the currency at 0 index of X
train, val, test = [TimeSeriesDataset(X[0], y[0], dtype) for dtype in ['train', 'val', 'test']]